In [1]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [2]:
get_ipython().run_cell_magic('capture', '', "\n#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n\n!python3 -m pip install pandas\n")

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [3]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

- DEFINE YOUR CLASSES AND FUNCTIONS 
-----------------------------------
This is not required, but is helpful in keeping your notebook organized. 
You can use the following cell or several cells to define your functions
and classes to keep them separate from your analysis or results code.
In general it useful to define your methods in a separate cell from where
it is run.

In [4]:
def example_function():
    print('Hello World')

In [5]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [6]:
def get_data(table_name):
    query = f"""
    SELECT *
    FROM `ironhacks-data.ironhacks_competition.{table_name}`
    """

    # QUERY THE DATA ONCE
    query_job = bigquery_client.query(query)
    return query_job.to_dataframe()

In [7]:
unemploy = get_data('unemployment_data')

In [8]:
wage = get_data('wage_data')

In [9]:
sub = get_data('prediction_list')

In [10]:
pred = unemploy.groupby('uu_id')['total_claims'].mean().apply(lambda x: int(x))

In [11]:
pred = unemploy.groupby('uu_id').apply(lambda x: x.sort_values('week_number').drop_duplicates('week_number')['total_claims'].ewm(alpha=1/3).mean().to_numpy()[-1])

In [12]:
pred = pred[sub['uu_id'].to_list()]

In [13]:
df = pd.DataFrame({
    'uu_id': pred.index,
    'week_number': 39,
    'total_claims': pred.to_numpy().astype(np.int32)
})

In [14]:
df

,uu_id,week_number,total_claims
0,5bf51fc2e162d6faf9e3cf79e4198378,39,16
1,420b44cc7e3f55d738df565421e59941,39,12
2,e39c66ecceec76ee8f9f811fa4a2d246,39,12
3,a90462cd11ae4e43144239bf7c4828a4,39,15
4,8b20a6749088c7ff1237983076ebfeaa,39,13
...,...,...,...
520,46c4f6c75e663b1ca82ea7994e6d83d3,39,37
521,1deebda501712e7595b531b8337bc31a,39,22
522,5a9758f65f001b6432ff31ff64a459d7,39,14
523,e8b3b95e93a6dc7dbb90f4e72e7ac065,39,13


In [15]:
df.to_csv('submission_prediction_output.csv', index=False)